# ETL Pipeline for Pterodactyl Game Servers Logs

### Index

- Install requierements
- Import libraries and setup key variables
- Setup directories, functions and folder creation
- Extract logs from each active Minecraft Server
- Transformation from logs data into information
- Load processed data into Data Warehouse (Postgres)

## Install requierements

In [ ]:
pip install -r requirements.txt

## Import libraries and setup key variables
Remember to add you own credentials in the .env file for them to be loaded here

In [ ]:
import urllib.request, zipfile, requests, gzip, csv, os, re
from sqlalchemy import create_engine, text
from pydactyl import PterodactylClient
from dotenv import load_dotenv
import pandas as pd

# Load .env file credentials
load_dotenv()

# Database connection
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
database = os.getenv('POSTGRES_DATABASE')
username = os.getenv('POSTGRES_USERNAME')
password = os.getenv('POSTGRES_PASSWORD')
connection = f'postgresql://{username}:{password}@{host}:{port}/{database}'

# Pterodactyl connection
pterodactyl_url = os.getenv('PTERODACTYL_URL')
application_api_key = os.getenv('PTERODACTYL_APP_KEY')
client_api_key = os.getenv('PTERODACTYL_CLI_KEY')

# Connecto to Pterodactyl Application API
api_app = PterodactylClient(pterodactyl_url, application_api_key, debug=False)
# Connecto to Pterodactyl Client API
api_cli = PterodactylClient(pterodactyl_url, client_api_key, debug=False)

## Setup directories, functions and folder creation

In [ ]:
# Setup directories
pwd = os.getcwd() #os.path.dirname(os.path.realpath(__file__)) this is used for .py files
raw_logs_folder = os.path.join(pwd, 'raw_logs')

# Create new folder if not exists
def mkdir(folder_dir):
    if not os.path.exists(folder_dir):
        os.makedirs(os.path.join(pwd, folder_dir))

# Sort a list of logs names
def sort_list_logs(logs):
    def logs_modifications(log):
        # remove the '-' from the log
        date_part, number_part = log.split('-')[:3], log.split('-')[3]
        # Join the parts of date to transform
        date_log = '-'.join(date_part)
        # remove the '.log' extension and pass the number to int
        number_log = int(number_part.split('.')[0])
        # return the date and number for sorting
        return date_log, number_log

    # use the logs_modifications function to sort the logs by date and number
    sorted_logs = sorted(logs, key=logs_modifications)
    return sorted_logs

# Get last index from a list when matching with an element
def last_index(list, element):
    try:
        return len(list) - 1 - list[::-1].index(element)
    except ValueError:
        raise ValueError(f"'{element}' not found in the list")

# Extract the compressed files (.gz and .zip)    
def extract_compressed_file(compressed_file_path, decompressed_folder_path):
    if compressed_file_path.endswith('.gz'):
        with gzip.open(compressed_file_path, 'rb') as compressed_file:
            with open(decompressed_folder_path, 'wb') as decompressed_file:
                decompressed_file.write(compressed_file.read())
    elif compressed_file_path.endswith('.zip'):
        with zipfile.ZipFile(compressed_file_path, 'r') as zip_file:
            zip_file.extractall(os.path.dirname(decompressed_folder_path))

# Create folder if not exist
mkdir(raw_logs_folder)

In [ ]:
def download_logs(identifier, egg, last_log, staging_folder):
    eggs_minecraft = ('Vanilla Minecraft', 'Forge Minecraft', 'Paper', 'Spigot', 'Mohist')
    eggs_ark = ('Ark: Survival Evolved',)
    all_eggs = eggs_minecraft + eggs_ark

    folder_logs = {
        'Vanilla Minecraft': '/logs/',
        'Forge Minecraft': '/logs/',
        'Paper': '/logs/',
        'Spigot': '/logs/',
        'Mohist': '/logs/',
        'Ark: Survival Evolved': '/ShooterGame/Saved/Logs/'
    }

    log_pattern = {
        'Vanilla Minecraft': r'^\d{4}-\d{2}-\d{2}-\d.*', # yyyy-mm-dd-n*
        'Forge Minecraft': r'^\d{4}-\d{2}-\d{2}-\d.*',
        'Paper': r'^\d{4}-\d{2}-\d{2}-\d.*',
        'Spigot': r'^\d{4}-\d{2}-\d{2}-\d.*',
        'Mohist': r'^\d{4}-\d{2}-\d{2}-\d.*',
        'Ark: Survival Evolved': r'^\d{4}-\d{2}-\d{2}-\d.*' #ServerGame.316.2023.11.17_19.25.10.log or ShooterGame-backup-2023.11.16-21.13.36.log - not sure 
    }

    compression_ext = ('.gz', '.zip')

    if egg in all_eggs:
        # Create a folder as staging area for each server
        folder_server_dir = os.path.join(staging_folder, identifier)
        mkdir(folder_server_dir)

        # Get the last log date to download only necessary logs
        if last_log is not None:
            last_log_date = last_log.strftime('%Y-%m-%d')
        else:
            # Set a default value to download all logs in case when last log date is not available
            last_log_date = '2000-01-01'

        # Get a list of the logs inside the server
        try:
            log_files = api_cli.client.servers.files.list_files(identifier, folder_logs[egg])
            log_files_data = log_files.get('data', [])
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404 or e.response.status_code == 504:
                print(f"Server {identifier} not found in Pterodactyl.")
                return None # Skip this and continue to the next iteration

        # If log_files_data is not empty, sort the list of logs based on the date naming
        if log_files_data:
            list_logs = [file['attributes']['name'] for file in log_files_data if re.match(log_pattern[egg], file['attributes']['name'])]
            sorted_list_logs = sort_list_logs(list_logs)
            sorted_list_logs_date = [item[:10] for item in sorted_list_logs]
        else:
            print(f"No log files found in server {identifier} with the client API.")

        # Select only a list of downloadable_logs which are new; after the last_log_date
        try:
            index_last_log = last_index(sorted_list_logs_date, last_log_date)
        except:
            index_last_log = -1
        downloadable_logs = sorted_list_logs[index_last_log + 1:]

        # Download all logs in the list downloadable_logs
        list_download = [api_cli.client.servers.files.download_file(server_info[0], f'/logs/{log}') for log in downloadable_logs]
        if list_download:
            [urllib.request.urlretrieve(list_download[i], os.path.join(folder_server_dir, list_logs[i])) for i in range(len(list_download))]
        print(f'Files downloaded: {len(list_download)}')

        # Uncompressing files if needed
        for filename in os.listdir(folder_server_dir):
            if filename.endswith(compression_ext):
                compressed_file_path = os.path.join(folder_server_dir, filename)
                decompressed_file_path = os.path.splitext(compressed_file_path)[0]  # Remove the last extension

                # Uncompress the file
                extract_compressed_file(compressed_file_path, decompressed_file_path)

                # Delete the compressed file
                os.remove(compressed_file_path)

## Extract logs from each active Minecraft Server

In [ ]:
SCHEMA_PTERODACTYL = 'pterodactyl'
SCHEMA_MINECRAFT = 'minecraft'

eggs_ready = ('Vanilla Minecraft', 'Forge Minecraft', 'Paper', 'Mohist') # Vanilla Bedrock is still not ready to be processed

# Get server information
engine = create_engine(connection)
with engine.connect() as conn:
    list_servers = conn.execute(text(f'SELECT servers.identifier, eggs.name, last_log_date.last_date FROM {SCHEMA_PTERODACTYL}.servers JOIN {SCHEMA_PTERODACTYL}.eggs ON eggs.id = servers.egg_id LEFT JOIN {SCHEMA_MINECRAFT}.last_log_date ON last_log_date.server_identifier = servers.identifier WHERE servers.is_active = true and servers.nest_id = 1'))

for server_info in list_servers:

    download_logs(identifier = server_info[0], egg = server_info[1], last_log = server_info[2], staging_folder = raw_logs_folder)

## Transformation from logs data to information

In [17]:
log_extension = '.log'

folder = os.listdir(raw_logs_folder)[0]
folder_server_dir = os.path.join(raw_logs_folder, folder)
folder_server_dir

log_files = [log for log in os.listdir(folder_server_dir) if log.endswith(log_extension)]
log_files = sort_list_logs(log_files)

if log_files:

    all_logs = ""
    for log_file in log_files:
        with open(os.path.join(folder_server_dir, log_file), 'r', encoding='utf-8') as file:
            log_contents = file.read().split('\n')
            log_contents = "\n".join([f'[{log_file[:-(2+len(log_extension))]}] ' + line for line in log_contents if line.strip() != ""])
            all_logs += log_contents + "\n"
all_logs


'[2023-11-14] [23:58:16] [main/INFO] [STDOUT/]: Update in check...\n[2023-11-14] [23:58:16] [main/INFO] [STDOUT/]: If you don\'t want to enable update check, set check_update in mohist-config/mohist.yml to false\n[2023-11-14] [23:58:18] [main/INFO] [STDOUT/]: No update found, latest version: (1.16.5-1187) current version: (1.16.5-1187)\n[2023-11-14] [23:58:18] [main/INFO] [STDOUT/]: You need to accept the eula to launch your server. Type true to continue.\n[2023-11-14] [23:58:29] [main/INFO] [cpw.mods.modlauncher.Launcher/MODLAUNCHER]: ModLauncher running: args [--gameDir, ., --launchTarget, fmlserver, --fml.mohistVersion, 1187, --fml.forgeVersion, 36.2.39, --fml.mcpVersion, 20210115.111550, --fml.mcVersion, 1.16.5, --fml.forgeGroup, com.mohistmc, pause]\n[2023-11-14] [23:58:29] [main/INFO] [cpw.mods.modlauncher.Launcher/MODLAUNCHER]: ModLauncher mohist-8.1.3 starting: java version 18.0.2.1 by Eclipse Adoptium\n[2023-11-14] [23:58:29] [main/INFO] [net.minecraftforge.fml.loading.FixSSL/

In [16]:
import re
re.findall(r'\[(\d{4}-\d{2}-\d{2})\] \[.*?(\d{2}:\d{2}:\d{2})(?: INFO|)\](?: [.*//INFO]:|:) (.*)', '[2023-01-01] [21:29:59 INFO]: good\n[2023-02-02] [14:38:37] [asdf/INFO]: good')

[('2023-01-01', '21:29:59', 'good')]

In [ ]:
df_all_logs = pd.DataFrame(columns=['server_identifier', 'date', 'time', 'information', 'user', 'activity'])

for folder in os.listdir(raw_logs_folder): 
    folder_server_dir = os.path.join(raw_logs_folder, folder)

    # Read all logs as one
    log_files = [log for log in os.listdir(folder_server_dir) if log.endswith('.log')]
    log_files = sort_list_logs(log_files)

    if log_files:

        all_logs = ""
        for log_file in log_files:
            with open(os.path.join(folder_server_dir, log_file), 'r', encoding='utf-8') as file:
                log_contents = file.read().split('\n')
                log_contents = "\n".join([f'[{log_file[:-(2+len(extension_file_uncompessed))]}] ' + line for line in log_contents if line.strip() != ""])
                all_logs += log_contents + "\n"

        # Transform information in meaningful information
        pattern = r'\[(\d{4}-\d{2}-\d{2})\] \[.*?(\d{2}:\d{2}:\d{2}).*?\] \[(.*?)/.*?\]: (.*?)\n'
        matches = re.findall(pattern, all_logs)

        # Create a list of dictionaries to store the extracted data
        log_data = [{'server_identifier': folder, 'date': match[0], 'time': match[1], 'category': match[2], 'information': match[3]} for match in matches]
        # Create a dataframe of the logs
        df_logs = pd.DataFrame(log_data)
        df_logs['user'] = None
        df_logs['activity'] = None

        # Filter by column category selecting only Server thread logs nad delete that column
        df_logs = df_logs[df_logs['category'] == 'Server thread'][['server_identifier', 'date', 'time', 'information', 'user', 'activity']]

        # Create a dataframe of the users
        df_users = pd.read_csv(os.path.join(folder_server_dir,'users.csv'))

        # Add information when server started
        index=df_logs[df_logs['information'].str.startswith("Starting minecraft server version") | df_logs['information'].str.startswith("Starting minecraft server on")].index
        if not index.empty:
            df_logs.loc[index, 'user'] = 'server'
            df_logs.loc[index, 'activity'] = 'start'
        # Add information when server stopped
        index=df_logs[df_logs['information'].str.startswith("Stopping server") | df_logs['information'].str.startswith("Stopping the server")].index
        if not index.empty:
            df_logs.loc[index, 'user'] = 'server'
            df_logs.loc[index, 'activity'] = 'stop'
        # Add information when user did something
        for user in df_users['name']:
            # Add information when user activity
            index=df_logs[df_logs['information'].str.startswith(user)].index
            if not index.empty:
                df_logs.loc[index, 'user'] = user
                df_logs.loc[index, 'activity'] = 'action'
            # Add information when user login
            index=df_logs[df_logs['information'].str.startswith(f'{user} joined the game')].index
            if not index.empty:
                df_logs.loc[index, 'user'] = user
                df_logs.loc[index, 'activity'] = 'login'
            # Add information when user logout
            index = df_logs[df_logs['information'].str.startswith(f'{user} left the game')].index
            if not index.empty:
                df_logs.loc[index, 'user'] = user
                df_logs.loc[index, 'activity'] = 'logout'
            # Delete duplicated activity of login and logout
            index=df_logs[df_logs['information'].str.startswith(f'{user}[') | df_logs['information'].str.startswith(f'{user} lost connection: Disconnected')].index
            if not index.empty:
                df_logs.loc[index, 'user'] = None
                df_logs.loc[index, 'activity'] = None
        # Rows with no server/user activity is deleted
        df_logs = df_logs.dropna(subset=['activity'])

        # Save every log in df_all_logs for each iteration
        df_all_logs = pd.concat([df_all_logs, df_logs], ignore_index=True)

# Delete all logs inside each server folder
for folder in os.listdir(raw_logs_folder):
    folder_server_dir = os.path.join(raw_logs_folder, folder)
    [os.remove(os.path.join(folder_server_dir, log)) for log in os.listdir(folder_server_dir) if log.endswith(extension_file_uncompessed)]

## Load processed data into Data Warehouse (Postgres)

In [ ]:
# Set Schema and Table names in Postgres
SCHEMA = 'pterodactyl'
TABLE = 'activity'

# Connect to database and upload all new logs into table
engine = create_engine(connection)
with engine.connect() as conn:

# Start a new transaction
    trans = conn.begin()

    try:
        # Load all new activity into postgres
        df_all_logs.to_sql(name = TABLE, schema = SCHEMA, con = conn, if_exists='append', index=False)
        # Commit the transaction
        trans.commit()

    except Exception as e:
        # Rollback the transaction on exception
        print('!!! [ERROR IN DATABASE QUERIES] !!!')
        trans.rollback()
        print('Transaction has been rolled back')
        print(f'Error occurred during transaction:\n{e}')
        raise